In [1]:
import pandas as pd 
import numpy as np 
from scipy import stats 


In [7]:
df = pd.read_csv("C:/Users/Abhi/Desktop/Python_Data_Analytics_Projects/behavioral-bias-detector/data/processed/TCS_panic_signals.csv")
df["date"]= pd.to_datetime(df["date"])
df = df.sort_values("date").reset_index(drop=True)
df.head()

,date,close,high,low,open,volume,daily_return,return_5d,return_10d,volatility_10d,rolling_max,drawdown,volume_mean_20d,volume_std_20d,volume_zscore,high_52w,dist_from_52w_high,panic_sell
0,2020-01-13,1919.979858,1945.049432,1915.027149,1944.085341,2843893,-0.010481,-0.004590,0.003275,0.012769,1989.490601,-0.034939,3603601.00,1.934266e+06,-0.392763,1989.490601,-0.034939,False
1,2020-01-14,1934.487061,1954.560484,1924.056045,1924.056045,2948452,0.007556,0.000476,0.020910,0.012398,1989.490601,-0.027647,3472314.75,1.881876e+06,-0.278373,1989.490601,-0.027647,False
2,2020-01-15,1952.018066,1955.612060,1923.354499,1939.833926,2620681,0.009062,-0.012571,0.027357,0.012592,1989.490601,-0.018835,3300365.15,1.787783e+06,-0.380183,1989.490601,-0.018835,False
3,2020-01-16,1962.448975,1971.389873,1941.586736,1952.061619,3117214,0.005344,0.011041,0.037610,0.012335,1989.490601,-0.013592,3043671.80,1.355948e+06,0.054237,1989.490601,-0.013592,False
4,2020-01-17,1945.181641,1975.379203,1939.834519,1964.159150,3281059,-0.008799,0.002508,0.008384,0.011468,1989.490601,-0.022272,2895149.50,1.129934e+06,0.341533,1989.490601,-0.022272,False


In [9]:
loss_days = df[df["daily_return"]<0]
gain_days = df[df["daily_return"]>0]
len(loss_days),len(gain_days)

(592, 636)

In [13]:
t_stat,p_value = stats.ttest_ind(
    loss_days["volume_zscore"],
    gain_days["volume_zscore"],
    equal_var=False
)
t_stat,p_value
    

(-0.3750497334982623, 0.7076893887848722)

In [17]:
if p_value < 0.05:
    print("Statistically significant loss aversion detected")
else:
    print("No strong evidence of loss aversion")


No strong evidence of loss aversion


In [19]:
mean_diff = (
    loss_days["volume_zscore"].mean() -
    gain_days["volume_zscore"].mean()
)

pooled_std = np.sqrt(
    (loss_days["volume_zscore"].var() + gain_days["volume_zscore"].var()) / 2
)

cohens_d = mean_diff / pooled_std
cohens_d


-0.021440055515852336

In [27]:
stress_days = df[df["drawdown"] < -0.05]   # more than 5% below peak
normal_days = df[df["drawdown"] >= -0.05]

In [29]:
u_stat, p_val = stats.mannwhitneyu(
    stress_days["volume"],
    normal_days["volume"],
    alternative="two-sided"
)

u_stat, p_val

(134508.0, 4.592727981573803e-08)

In [31]:
if p_val < 0.05:
    print("Overtrading during drawdowns detected")
else:
    print("No strong overtrading evidence")

Overtrading during drawdowns detected


In [33]:
panic_days = df[df["panic_sell"] == True]
normal_loss_days = df[
    (df["daily_return"] < 0) & (df["panic_sell"] == False)
]


In [35]:
stats.ttest_ind(
    panic_days["daily_return"],
    normal_loss_days["daily_return"],
    equal_var=False
)

TtestResult(statistic=-6.481910380240657, pvalue=7.111885757968789e-06, df=16.206064183792897)

In [39]:
summary = pd.DataFrame({
    "metric": [
        "Loss vs Gain Volume (p-value)",
        "Loss Aversion Effect Size",
        "Overtrading p-value"
    ],
    "value": [
        p_value,
        cohens_d,
        p_val
    ]
})
summary


,metric,value
0,Loss vs Gain Volume (p-value),7.076894e-01
1,Loss Aversion Effect Size,-2.144006e-02
2,Overtrading p-value,4.592728e-08


In [41]:
df.to_csv("C:/Users/Abhi/Desktop/Python_Data_Analytics_Projects/behavioral-bias-detector/data/processed/TCS_bias_test_summary.csv",index = False)